# IANNwTF HW 4
## Group 10

The following contains our solution to the exercises in IANNwTF HW 04. A Jupyter notebook versus a module format was chosen this time for purposes of organization.

### Assigment 1: Reviews
We review the homeworks for Groups 15 and 32.

### Assignment 2: MNIST Math

### 2.1 Preparing the MNIST Math Dataset

In [25]:
# Needed Imports
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense
import tqdm
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [26]:
# 2.1 Load Dataset
(train_ds, test_ds), ds_info = tfds.load ('mnist', split =['train', 'test'], as_supervised = True, with_info = True)

# Info on the dataset (refresher)
# print("ds_info: \n", ds_info)
# tfds.show_examples(train_ds, ds_info)

In [111]:
# 2.2 Data Pipeline
def prepare_data(dataset, batchsize):

    '''
    :param dataset: the dataset to be prepared for input into the network
    :param batchsize: the desired batchsize
    :return: 2 datasets, one each for each of the math problems defined (see below), created after the original database was preprocessed with the
    steps below
    '''

    # Step One - General Preprocessing

    # convert data from uint8 to float32
    dataset = dataset.map(lambda img, target: (tf.cast(img, tf.float32), target))

    # flatten the images into vectors
    dataset = dataset.map(lambda img, target: (tf.reshape(img, (-1,)), target))

    # input normalization, just bringing image values from range [0, 255] to [-1, 1]
    dataset = dataset.map(lambda img, target: ((img / 128.) - 1., target))

    # Step 2 - Pairing Data Tuples & Respective Parameterized Targets

    # create a dataset that contains 2000 samples from the overall dataset paired with 2000 other samples
    data = tf.data.Dataset.zip((dataset.shuffle(2000), dataset.shuffle(2000)))

    # create the dataset for the first math problem (a + b >= 5) - remembering to cast to int versus boolean!
    greateqfive = data.map(lambda x1, x2: (x1[0], x2[0], x1[1]+x2[1]>=5))
    greateqfive = greateqfive.map(lambda x1, x2, t: (x1, x2, tf.cast(t, tf.int32)))

    # create the dataset for the second math problem (a - b = y)
    subtr = data.map(lambda x1, x2: (x1[0], x2[0], x1[1]-x2[1]))

    # Step 3 - Batching & Prefetching

    # run batching and prefetching for both datasets
    greateqfive = greateqfive.batch(batchsize)
    greateqfive = greateqfive.prefetch(tf.data.AUTOTUNE)
    subtr = subtr.batch(batchsize)
    subtr = subtr.prefetch(tf.data.AUTOTUNE)

    # return BOTH datasets
    return greateqfive, subtr

In [112]:
# Check data pipeline by examining one example from each of the four created datasets (one for each math problem for train and test)

train_ds_gef, train_ds_subtr = prepare_data(train_ds, batchsize = 32)
test_ds_gef, test_ds_subtr = prepare_data(test_ds, batchsize = 32)

for img1, img2, label in train_ds_gef.take(1):
    print(img1.shape, img2.shape, label.shape)

for img1, img2, label in train_ds_subtr.take(1):
    print(img1.shape, img2.shape, label.shape)

for img1, img2, label in test_ds_gef.take(1):
    print(img1.shape, img2.shape, label.shape)

for img1, img2, label in test_ds_subtr.take(1):
    print(img1.shape, img2.shape, label.shape)


(32, 784) (32, 784) (32,)
(32, 784) (32, 784) (32,)
(32, 784) (32, 784) (32,)
(32, 784) (32, 784) (32,)


In [113]:
# Initiate the logs and metrics
config_name= "config_name"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{config_name}/{current_time}/train"
val_log_path = f"logs/{config_name}/{current_time}/val"

# log writer for training metrics
train_summary_writer = tf.summary.create_file_writer(train_log_path)

# log writer for validation metrics
val_summary_writer = tf.summary.create_file_writer(val_log_path)

# Initiate epochs and learning rate as global variables
epochs = 5
learning_rate = 0.1 # 0.01

# Define arrays for saving values for later visualization
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

### Assignment 3: Building Shared Weight Models

In [121]:
class MyModel(tf.keras.Model):
    def __init__(self, numlayers, subtask, optimizer):

        '''
        param: numlayers - the desired number of hidden layers
        param: subtask - the subtask the network is being asked to solve (relevant for output layer)
        param: optimizer - the optimizer to be used
        '''

        super(MyModel, self).__init__()

        self.optimizer = optimizer
        self.subtask = subtask

        self.layerlist = [Dense(layers,activation="relu") for layers in range(numlayers)]

        if subtask == 0:
            self.output_layer = Dense(units=1, activation=tf.nn.sigmoid)
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
        elif subtask == 1:
            self.output_layer = Dense(units=1, activation=tf.nn.softmax) # not 10 units, since the label.shape is (32,) not (32,10)
            self.loss_function = tf.keras.losses.MeanSquaredError()

        self.metrics_list = [
                    tf.keras.metrics.Mean(name="loss"),
                    tf.keras.metrics.BinaryAccuracy(name="acc"), # only for subtask 0, not for subtask 1
                    #tf.keras.metrics.TopKCategoricalAccuracy(3,name="top-3-acc")
                    ]

    @tf.function
    def __call__(self, input: tuple, training = False):

        # feed both inputs separately into the first layer, then concatenate the results before passing activity through the rest of the network UP TO the output layer
        for layer in range(len(self.layerlist)):
             if layer == 0:
                 i1 = self.layerlist[layer](input[0])
                 i2 = self.layerlist[layer](input[1])
                 i = tf.concat([i1, i2], axis=1)   # e.g. axis=1: (32,784) + (32,784) -> (32, 1568)
             else:
                 i = self.layerlist[layer](i)

        # run the activity through the output layer after it passes through the hidden layers
        output = self.output_layer(i)

        return output

    @property
    def metrics(self):
        return self.metrics_list

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

    @tf.function
    def train_step(self, input):
        img1, img2, label = input

        with tf.GradientTape() as tape:
            prediction = self((img1, img2), training=True)
            loss = self.loss_function(label, prediction)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update loss metric
        self.metrics[0].update_state(loss)

        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics[1:]:
            metric.update_state(label, prediction) # + tf.reduce_sum(self.losses)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    @tf.function
    def test_step(self, input):

        img1, img2, label = input

        prediction = self((img1, img2), training=False)
        loss = self.loss_function(label, prediction) # + tf.reduce_sum(self.losses)

        # update loss metric
        self.metrics[0].update_state(loss)

        # for accuracy metrics:
        for metric in self.metrics[1:]:
            metric.update_state(label, prediction)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


### Assignment 4: Training the Networks

In [130]:
def training(numlayers, subtask, optimizer):
    '''
    :param: numlayers: the number of layers desired in the network to be trained
    :param subtask: defines the subtask to be solved, 0 is a + b >= 5, 1 is a - b = y
    :param optimizer: the optimizer function to use
    :return:
    '''

    # Initiate a model with the requested parameters
    network = MyModel(numlayers, subtask, optimizer)

    # Initialize the train and test datasets, and the loss function, based on the subtask
    if subtask == 0:
        train = train_ds_gef
        test = test_ds_gef

    else:
        train = train_ds_subtr
        test = test_ds_subtr

    # Train the model
    for epoch in range(epochs):
        print(f"Epoch {epoch}")

        for data in tqdm.tqdm(train, position=0, leave=True):
            metrics = network.train_step(data)

            with train_summary_writer.as_default():
                for metric in network.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # Piper Attempt at Retrieve Mets for Vis
        #train_losses.append(network.metrics[0].result())
        #train_accuracies.append(network.metrics[1].result())

        # print the end acc and loss
        print([f"train_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # wooki's struggle to make a list for the visualization (pls don't delete)
        #print(metrics)
        #train_losses.append(metrics.values.numpy()[1])
        #train_accuracies.append(metrics.values.numpy()[1])
        # for (key, value) in metrics:

            # if key == "loss":
            #     train_losses.append(value.numpy())
            # elif key == "acc":
            #     train_accuracies.append(value.numpy())
            
        # pls don't delete until here

        # reset metrics for next round
        network.reset_metrics()

        # Testing
        for data in test:
            metrics = network.test_step(data)

            # log the accs and losses
            with val_summary_writer.as_default():
                for metric in network.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step = epoch)

        # Piper Attempt at Retrieve Mets for Vis
        #test_losses.append(network.metrics[2].result())
        #test_accuracies.append(network.metrics[3].result())

        # print the end acc and loss
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics
        network.reset_metrics()

        #return train_losses,train_accuracies,test_losses,test_accuracies


In [132]:
# Train a model to solve the first math problem
training(2, 0, tf.keras.optimizers.Adam())
#training(2, 0, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.0))
#training(2, 0, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.5))
#training(2, 0, tf.keras.optimizers.RMSprop())
#training(2, 0, tf.keras.optimizers.Adagrad())

Epoch 0


100%|██████████| 1875/1875 [00:09<00:00, 207.85it/s]


['train_loss: 0.5251139402389526', 'train_acc: 0.8405666947364807']
['val_loss: 0.457953542470932', 'val_acc: 0.835099995136261']
Epoch 1


100%|██████████| 1875/1875 [00:07<00:00, 243.38it/s]


['train_loss: 0.44517287611961365', 'train_acc: 0.8381500244140625']
['val_loss: 0.4467918574810028', 'val_acc: 0.8356999754905701']
Epoch 2


...

['train_loss: 0.4387131631374359', 'train_acc: 0.8406000137329102']
['val_loss: 0.44238734245300293', 'val_acc: 0.8381999731063843']
Epoch 3


100%|██████████| 1875/1875 [00:08<00:00, 225.87it/s]


['train_loss: 0.44103941321372986', 'train_acc: 0.8391833305358887']
['val_loss: 0.44690239429473877', 'val_acc: 0.8356000185012817']
Epoch 4


100%|██████████| 1875/1875 [00:07<00:00, 236.95it/s]


['train_loss: 0.43913599848747253', 'train_acc: 0.8403333425521851']
['val_loss: 0.43809232115745544', 'val_acc: 0.8410000205039978']


In [133]:
# Train a model to solve the second math problem
training(2, 1, tf.keras.optimizers.Adam())
#training(2, 1, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.0))
#training(2, 1, tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.5))
#training(2, 1, tf.keras.optimizers.RMSprop())
#training(2, 1, tf.keras.optimizers.Adagrad())

Epoch 0


100%|██████████| 1875/1875 [00:09<00:00, 192.42it/s]


['train_loss: 17.72833251953125', 'train_acc: 0.08996666967868805']
['val_loss: 17.72484016418457', 'val_acc: 0.08820000290870667']
Epoch 1


100%|██████████| 1875/1875 [00:07<00:00, 235.68it/s]


['train_loss: 17.66116714477539', 'train_acc: 0.08966666460037231']
['val_loss: 17.768171310424805', 'val_acc: 0.09130000323057175']
Epoch 2


...

['train_loss: 17.676265716552734', 'train_acc: 0.08996666967868805']
['val_loss: 17.715755462646484', 'val_acc: 0.09179999679327011']
Epoch 3


100%|██████████| 1875/1875 [00:07<00:00, 236.25it/s]


['train_loss: 17.716867446899414', 'train_acc: 0.09125000238418579']
['val_loss: 17.947484970092773', 'val_acc: 0.08609999716281891']
Epoch 4


100%|██████████| 1875/1875 [00:08<00:00, 228.93it/s]


['train_loss: 17.70676612854004', 'train_acc: 0.09099999815225601']
['val_loss: 17.95946502685547', 'val_acc: 0.08799999952316284']


**Accuracy does initially seem to go up over the epochs, but then drops again. But then this is the harder task I think. Visualization should be a matter of pulling out the metric results into the lists we created so we can use those lists for the next part. I tried this with the code I added above (marked as my attempt), but no luck.**

### Assignment 5 - Experiments

Run training w/ classic SGD (no momentum)

Run training w/ Adam

Run training w/ SGD + Momentum

Run training w/ RMSrop

Run training w/ AdaGrad

In [ ]:
# Visualize the results of the above training runs

# NEED TO BE WORKED ON
fig, axs = plt.subplots(5) 
fig.suptitle('Vertically stacked subplots')
x = np.linspace(0, epochs)
y = np.sin(x ** 2)
ax1.plot(x, y)
ax2.plot(x, -y)

line1, = plt.plot(train_losses)
line2, = plt.plot(test_losses)
line3, = plt.plot(train_accuracies)
line4, = plt.plot(test_accuracies)
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend((line1, line2, line3, line4),("Training Loss", "Test Loss", "Training Accuracy", "Test Accuracy"))
fig.savefig("Title-Of-The-Figure")
plt.show()
